<a href="https://colab.research.google.com/github/kamaleshpantra/StockPulse/blob/main/stock_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache-flink
!pip install praw
!pip install yfinance
!pip install streamlit
!pip install pyngrok
!pip install nltk
!pip install pandas
!pip install matplotlib
!pip install plotly
!pip install scikit-learn
!pip install tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.6 MB/s eta 0:00:00


In [ ]:
# Cell 1: Imports and Setup
import praw
import yfinance as yf
from pyflink.table import EnvironmentSettings, TableEnvironment
from pyflink.table.udf import udf
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import pandas as pd
import json
from datetime import datetime, timedelta
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import plotly.graph_objects as go
import streamlit as st
from pyngrok import ngrok

nltk.download('vader_lexicon')  # Download VADER lexicon

# Define sectors
sectors = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "TSLA"],
    "Finance": ["JPM", "BAC", "WFC", "GS"],
    "Healthcare": ["PFE", "JNJ", "MRK", "GILD"]
}

# Reddit credentials (replace with yours)
reddit = praw.Reddit(client_id="L95_I9ZkZOh5XLWN61EzSg",
                     client_secret="GpUVeC1rBxtbA7JBy9_Bf7OlkwabuQ",
                     user_agent="stock_market")

# VADER setup
sid = SentimentIntensityAnalyzer()

print("Setup complete!")

Setup complete!


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
# Cell 2: Fetch Reddit Data
def fetch_large_reddit(sector, subreddits=["wallstreetbets", "stocks", "investing"], limit_per_query=1000):
    companies = sectors[sector]
    all_posts = []
    for company in companies:
        for subreddit in subreddits:
            print(f"Fetching {company} from r/{subreddit}...")
            try:
                for submission in reddit.subreddit(subreddit).search(company, limit=limit_per_query):
                    post = {
                        "company": company,
                        "text": submission.title + " " + submission.selftext,
                        "timestamp": submission.created_utc,
                        "subreddit": subreddit
                    }
                    all_posts.append(post)
            except Exception as e:
                print(f"Error for {company} in {subreddit}: {e}")
            time.sleep(2)  # Rate limit delay
    filename = f"reddit_{sector}_large.jsonl"
    with open(filename, "w") as f:
        for post in all_posts:
            f.write(json.dumps(post) + "\n")
    print(f"Saved {len(all_posts)} posts to {filename}")
    return filename

# Input: Choose a sector
sector = "Technology"  # Replace with "Finance" or "Healthcare" to test others
reddit_file = fetch_large_reddit(sector)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Fetching AAPL from r/wallstreetbets...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Fetching AAPL from r/stocks...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



...

Fetching TSLA from r/stocks...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Fetching TSLA from r/investing...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Saved 2703 posts to reddit_Technology_large.jsonl


In [ ]:
# Cell 3: Fetch Stock Data
def fetch_large_stock(sector, years_back=1):
    companies = sectors[sector]
    end_date = datetime.now()
    start_date = end_date - timedelta(days=years_back * 365)
    for company in companies:
        print(f"Fetching stock data for {company}...")
        try:
            stock = yf.Ticker(company)
            hist = stock.history(start=start_date, end=end_date)
            hist.to_csv(f"stock_{company}_large.csv")
            print(f"Saved {len(hist)} days for {company}")
        except Exception as e:
            print(f"Error for {company}: {e}")

# Use same sector as Reddit
fetch_large_stock("Technology")

Fetching stock data for AAPL...
Saved 249 days for AAPL
Fetching stock data for MSFT...
Saved 249 days for MSFT
Fetching stock data for GOOGL...
Saved 249 days for GOOGL
Fetching stock data for TSLA...
Saved 249 days for TSLA


In [ ]:
# Cell 4: Preprocess Data with Flink (Fixed)
from pyflink.table.expressions import col

def preprocess_reddit(sector):
    @udf(result_type="STRING")
    def get_sentiment(text):
        score = sid.polarity_scores(text)["compound"]
        return "positive" if score > 0.05 else "negative" if score < -0.05 else "neutral"

    env_settings = EnvironmentSettings.in_batch_mode()
    table_env = TableEnvironment.create(env_settings)
    table_env.create_temporary_function("get_sentiment", get_sentiment)

    with open(f"reddit_{sector}_large.jsonl", "r") as f:
        data = [json.loads(line) for line in f]
    df = pd.DataFrame(data)
    table = table_env.from_pandas(df)
    result_table = table.select(
        col("company"),
        col("text"),
        col("timestamp"),
        get_sentiment(col("text")).alias("sentiment")
    )
    result_df = result_table.to_pandas()
    result_df["date"] = pd.to_datetime(result_df["timestamp"], unit="s").dt.date
    daily_sentiment = result_df.groupby(["company", "date"])["sentiment"].value_counts().unstack(fill_value=0)
    daily_sentiment.to_csv(f"daily_sentiment_{sector}.csv")
    print(f"Processed {len(result_df)} Reddit posts")

def preprocess_stock(sector):
    env_settings = EnvironmentSettings.in_batch_mode()
    table_env = TableEnvironment.create(env_settings)

    companies = sectors[sector]
    for company in companies:
        df = pd.read_csv(f"stock_{company}_large.csv")
        # Convert Date to datetime with UTC, then extract date
        if df["Date"].dtype == "object":  # If string, parse it
            df["date"] = pd.to_datetime(df["Date"], utc=True).dt.date
        else:  # If already datetime-like, just get date
            df["date"] = df["Date"].dt.date
        df["trend"] = (df["Close"].shift(-1) > df["Close"]).astype(int)  # 1 = up, 0 = down
        table = table_env.from_pandas(df[["date", "Close", "trend"]])
        table.to_pandas().to_csv(f"processed_stock_{company}_large.csv", index=False)
        print(f"Processed {len(df)} stock days for {company}")

# Run with your sector
sector = "Technology"  # Ensure this matches Cell 2/3
preprocess_reddit(sector)
preprocess_stock(sector)

Processed 2703 Reddit posts
Processed 249 stock days for AAPL
Processed 249 stock days for MSFT
Processed 249 stock days for GOOGL
Processed 249 stock days for TSLA


In [ ]:
# Cell 5: Prepare LSTM Data
def prepare_lstm_data(sector):
    companies = sectors[sector]
    X, y, dates = [], [], []
    scaler = MinMaxScaler()

    for company in companies:
        stock_df = pd.read_csv(f"processed_stock_{company}_large.csv")
        sentiment_df = pd.read_csv(f"daily_sentiment_{sector}.csv")
        sentiment_df = sentiment_df[sentiment_df["company"] == company]

        merged_df = pd.merge(stock_df, sentiment_df, on="date", how="inner")
        if len(merged_df) < 10:
            print(f"Skipping {company}: insufficient data")
            continue

        features = merged_df[["Close", "positive", "negative", "neutral"]].values
        features_scaled = scaler.fit_transform(features)
        target = merged_df["trend"].values

        seq_length = 10
        for i in range(len(features_scaled) - seq_length):
            X.append(features_scaled[i:i + seq_length])
            y.append(target[i + seq_length - 1])
            dates.append(merged_df["date"].iloc[i + seq_length - 1])

    return np.array(X), np.array(y), dates, scaler

X, y, dates, scaler = prepare_lstm_data(sector)
print(f"Prepared {len(X)} sequences for LSTM")

Prepared 327 sequences for LSTM


In [ ]:
# Cell 6: Train LSTM
def train_lstm(X, y):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2, verbose=1)
    return model

if len(X) > 0:
    model = train_lstm(X, y)
    print("LSTM training complete!")
else:
    print("No data for LSTM training")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.4896 - loss: 0.6934 - val_accuracy: 0.5909 - val_loss: 0.6840
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5228 - loss: 0.6896 - val_accuracy: 0.5909 - val_loss: 0.6785
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.5452 - loss: 0.6889 - val_accuracy: 0.5909 - val_loss: 0.6773
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5284 - loss: 0.6934 - val_accuracy: 0.5909 - val_loss: 0.6786
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.5733 - loss: 0.6830 - val_accuracy: 0.5909 - val_loss: 0.6770
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.5811 - loss: 0.6811 - val_accuracy: 0.5909 - val_loss: 0.6769
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.5531 - loss: 0.6877 - val_accuracy: 0.5758 - val_loss: 0.6807
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.5778 - loss: 0.6777 - val_accuracy: 0.5152 - val_loss: 0.6784


In [ ]:
# Cell 7: Predict Trends
def predict_trends(sector, model, scaler):
    X, _, dates, _ = prepare_lstm_data(sector)
    if len(X) == 0:
        return {company: "neutral" for company in sectors[sector]}

    predictions = model.predict(X)
    pred_labels = ["up" if p > 0.5 else "down" for p in predictions]

    latest_preds = {}
    companies = sectors[sector]
    for company in companies:
        company_dates = [d for d, pred in zip(dates, pred_labels) if company in pd.read_csv(f"daily_sentiment_{sector}.csv").loc[pd.read_csv(f"daily_sentiment_{sector}.csv")["date"] == d, "company"].values]
        if company_dates:
            latest_date = max(company_dates)
            latest_preds[company] = pred_labels[dates.index(latest_date)]
        else:
            latest_preds[company] = "neutral"
    return latest_preds

if len(X) > 0:
    predictions = predict_trends(sector, model, scaler)
    print("Predictions:", predictions)
else:
    predictions = {company: "neutral" for company in sectors[sector]}
    print("No predictions due to insufficient data")

11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
Predictions: {'AAPL': 'up', 'MSFT': 'up', 'GOOGL': 'up', 'TSLA': 'up'}


In [ ]:
# Cell 8: Validate Predictions
def validate_predictions(sector, predictions):
    actual = {}
    true, pred = [], []
    for company in sectors[sector]:
        stock_df = pd.read_csv(f"processed_stock_{company}_large.csv")
        if not stock_df.empty:
            latest_trend = "up" if stock_df["trend"].iloc[-2] == 1 else "down"
            actual[company] = latest_trend
            true.append(1 if latest_trend == "up" else 0)
            pred.append(1 if predictions.get(company, "neutral") == "up" else 0)

    metrics = {
        "accuracy": accuracy_score(true, pred),
        "precision": precision_score(true, pred, zero_division=0),
        "recall": recall_score(true, pred, zero_division=0),
        "f1": f1_score(true, pred, zero_division=0)
    }
    return actual, metrics

actual, metrics = validate_predictions(sector, predictions)
print("Actual:", actual)
print("Metrics:", metrics)

Actual: {'AAPL': 'up', 'MSFT': 'up', 'GOOGL': 'up', 'TSLA': 'up'}
Metrics: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}


In [ ]:
# Cell 9: Visualize Results
def plot_trends(sector, predictions, actual):
    figs = {}
    for company in sectors[sector]:
        df = pd.read_csv(f"processed_stock_{company}_large.csv")
        if not df.empty:
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=df["date"], y=df["Close"], mode="lines", name="Stock Price"))
            fig.add_annotation(x=df["date"].iloc[-2], y=df["Close"].iloc[-2],
                              text=f"Actual: {actual.get(company, 'N/A')}\nPred: {predictions.get(company, 'N/A')}",
                              showarrow=True)
            fig.update_layout(title=f"{company} Stock Trend (2 Years)", xaxis_title="Date", yaxis_title="Price")
            figs[company] = fig
            fig.show()

plot_trends(sector, predictions, actual)

In [ ]:
# Cell 10: Create and Deploy Streamlit App
st_script = """
# [Copy all imports and functions from Cells 1-9 here, replacing print() with st.write()]
import streamlit as st

st.title("Sector Stock Trend Predictor (LSTM)")

sector = st.selectbox("Select a Sector", list(sectors.keys()))

if st.button("Analyze"):
    with st.spinner("Fetching and processing..."):
        reddit_file = fetch_large_reddit(sector)
        fetch_large_stock(sector)
        preprocess_reddit(sector)
        preprocess_stock(sector)

        X, y, _, scaler = prepare_lstm_data(sector)
        if len(X) > 0:
            model = train_lstm(X, y)
            predictions = predict_trends(sector, model, scaler)
        else:
            predictions = {company: "neutral" for company in sectors[sector]}
            st.write("Not enough data for LSTM.")

        actual, metrics = validate_predictions(sector, predictions)
        figs = plot_trends(sector, predictions, actual)

        st.subheader("Predictions")
        for company, pred in predictions.items():
            st.write(f"{company}: Predicted {pred}, Actual {actual.get(company, 'N/A')}")

        st.subheader("Metrics")
        for metric, value in metrics.items():
            st.write(f"{metric.capitalize()}: {value:.2f}")

        st.subheader("Visualizations")
        for company, fig in figs.items():
            st.plotly_chart(fig)
"""

with open("app.py", "w") as f:
    f.write(st_script)

# Run in terminal: streamlit run app.py
print("Saved app.py. Run 'streamlit run app.py' in terminal, then use ngrok to deploy.")

Saved app.py. Run 'streamlit run app.py' in terminal, then use ngrok to deploy.
